In [77]:
import numpy 
import random
import math
import time
import matplotlib.pyplot as plt

In [78]:
def massCalculation(fit,PopSize,M):
    Fmax = max(fit) # calculate maximum value in the fit array
    Fmin = min(fit) # calculate minimum value in the fit array
    Fsum = sum(fit) # calculate sum of values in the fit array      
    Fmean = Fsum/len(fit) # mean value in the fit array
        
    if Fmax == Fmin:
        M = numpy.ones(PopSize)
    else:
        best = Fmin
        worst = Fmax
        
        for p in range(0,PopSize):
           M[p] = (fit[p]-worst)/(best-worst)
            
    Msum=sum(M)
    for q in range(0,PopSize):
        M[q] = M[q]/Msum
            
    return M

In [79]:
def gConstant(l,iters):
    alfa = 20
    G0 = 100
    Gimd = numpy.exp(-alfa*float(l)/iters)
    G = G0*Gimd
    return G

In [80]:
def gField(PopSize,dim,pos,M,l,iters,G,ElitistCheck,Rpower):
    final_per = 2
    if ElitistCheck == 1:
        kbest = final_per + (1-l/iters)*(100-final_per)
        kbest = round(PopSize*kbest/100)
    else:
        kbest = PopSize
            
    kbest = int(kbest)
    ds = sorted(range(len(M)), key=lambda k: M[k],reverse=True)
        
    Force = numpy.zeros((PopSize,dim))
    # Force = Force.astype(int)
    
    for r in range(0,PopSize):
        for ii in range(0,kbest):
            z = ds[ii]
            R = 0
            if z != r:                    
                x=pos[r,:]
                y=pos[z,:]
                esum=0
                imval = 0
                for t in range(0,dim):
                    imval = ((x[t] - y[t])** 2)
                    esum = esum + imval
                    
                R = math.sqrt(esum)
                
                for k in range(0,dim):
                    randnum=random.random()
                    Force[r,k] = Force[r,k]+randnum*(M[z])*((pos[z,k]-pos[r,k])/(R**Rpower+numpy.finfo(float).eps))
                    
    acc = numpy.zeros((PopSize,dim))
    for x in range(0,PopSize):
        for y in range (0,dim):
            acc[x,y]=Force[x,y]*G
    return acc

In [81]:
def move(PopSize,dim,pos,vel,acc):
    for i in range(0,PopSize):
        for j in range (0,dim):
            r1=random.random()
#           r2=random.random()
            vel[i,j]=r1*vel[i,j]+acc[i,j]
            pos[i,j]=pos[i,j]+vel[i,j]
    
    return pos, vel

In [82]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual=[]
        self.convergence = []
        self.optimizer=""
        self.objfname=""
        self.startTime=0
        self.endTime=0
        self.executionTime=0
        self.lb=0
        self.ub=0
        self.dim=0
        self.popnum=0
        self.maxiers=0

In [85]:
# from solution import solution
def GSA(objf,lb,ub,dim,PopSize,iters):
    # GSA parameters
    ElitistCheck =1
    Rpower = 1 
     
    s=solution()
        
    """ Initializations """
    
    vel=numpy.zeros((PopSize,dim))
    fit = numpy.zeros(PopSize)
    M = numpy.zeros(PopSize)
    gBest=numpy.zeros(dim)
    gBestScore=float("inf")
    
    pos=numpy.random.uniform(0,1,(PopSize,dim)) *(ub-lb)+lb
    
    convergence_curve=numpy.zeros(iters)
    
    print("GSA is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    s.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    
    for l in range(0,iters):
        for i in range(0,PopSize):
            l1 = [None] * dim
            l1=numpy.clip(pos[i,:], lb, ub)
            pos[i,:]=l1

            #Calculate objective function for each particle
            fitness=[]
            fitness=objf(l1)
            fit[i]=fitness
    
                
            if(gBestScore>fitness):
                gBestScore=fitness
                gBest=l1           
        
        """ Calculating Mass """
        M = massCalculation(fit,PopSize,M)

        """ Calculating Gravitational Constant """        
        G = gConstant(l,iters)        
        
        """ Calculating Gfield """        
        acc = gField(PopSize,dim,pos,M,l,iters,G,ElitistCheck,Rpower)
        
        """ Calculating Position """        
        pos, vel = move(PopSize,dim,pos,vel,acc)
        
        convergence_curve[l]=gBestScore
      
#         if (l%1==0):
#                print(['At iteration '+ str(l+1)+ ' the best fitness is '+ str(gBestScore)]);
    timerEnd=time.time()  
    s.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime=timerEnd-timerStart
    s.convergence=convergence_curve
    s.Algorithm="GSA"
    s.objectivefunc=objf.__name__

    return s


In [86]:
def F1(x):
  """ Sphere Function """
  s=numpy.sum(x**2);
  return s



def getFunctionDetails(a):
  # [name, lb, ub, dim]
  param = {  0: ["F1",-100,100,30],
            }
  return param.get(a, "nothing")

In [87]:
def selector(algo,func_details,popSize,Iter):
    function_name=func_details[0]
    lb=func_details[1]
    ub=func_details[2]
    dim=func_details[3]
    

    if(algo==0):
        x=GSA(F1,lb,ub,dim,popSize,Iter)    
    return x

In [89]:
Runs=30
PopSize = 50
iterations= 1000
iteration_array = range(1, iterations + 1)
mean_fitness_array = []


for k in range (0,Runs):
    func_details=getFunctionDetails(0)
    x=selector(0,func_details,PopSize,iterations)
#     print('x.Algorithm = ', x.Algorithm)
#     print('x.objectivefunc = ', x.objectivefunc)
#     print('x.startTime = ', x.startTime)
#     print('x.endTime = ', x.endTime)
#     print('x.executionTime = ', x.executionTime)
#     print('x.convergence = ', x.convergence)
    #plt.plot(iteration_array, x.convergence)  # Plot the chart
    mean_fitness = sum(x.convergence) / len(x.convergence)
    print("mean_fitness = ", mean_fitness)
    mean_fitness_array.append(mean_fitness)
    
    
#plt.show()  # display
print(mean_fitness_array)
avg_mean_fitness = sum(mean_fitness_array) / len(mean_fitness_array)
print(avg_mean_fitness)

GSA is optimizing  "F1"
mean_fitness =  198.88101600759745
GSA is optimizing  "F1"
mean_fitness =  165.0631295267853
GSA is optimizing  "F1"
mean_fitness =  179.86915079933624
GSA is optimizing  "F1"
mean_fitness =  193.2522408916877
GSA is optimizing  "F1"
mean_fitness =  163.32014185260903
GSA is optimizing  "F1"
mean_fitness =  193.29935173357987
GSA is optimizing  "F1"
mean_fitness =  194.18103619665828
GSA is optimizing  "F1"
mean_fitness =  204.91568544336513
GSA is optimizing  "F1"
mean_fitness =  189.44973767792933
GSA is optimizing  "F1"
mean_fitness =  174.61040749800316
GSA is optimizing  "F1"
mean_fitness =  157.41763359064657
GSA is optimizing  "F1"
mean_fitness =  176.78222521648377
GSA is optimizing  "F1"
mean_fitness =  190.35262145680355
GSA is optimizing  "F1"
mean_fitness =  202.5801483631368
GSA is optimizing  "F1"
mean_fitness =  164.62451829295827
GSA is optimizing  "F1"
mean_fitness =  149.26173774067408
GSA is optimizing  "F1"
mean_fitness =  204.4165099798962
G